In [32]:
import scipy.stats
import math
import pandas as pd

In [33]:
# import fragility curve parameters
vuln = pd.read_csv('function_params.csv')
vuln

,Country,Building cluster,Mean (tephra loading in kPa),Standard deviation,Total replacement cost (USD)
0,Indonesia,Weak,3.25,0.516,15400.0
1,Indonesia,Strong,7.53,0.529,37800.0
2,Philippines,Weak,3.60,0.482,9500.0
3,Philippines,Strong,8.83,0.498,20300.0
4,Aggregated,Weak,3.40,0.501,12800.0
5,Aggregated,Strong,7.80,0.522,34100.0
6,NaN,NaN,NaN,NaN,NaN


In [34]:
# check how to extract p-values from a cdf where the mean (loc) and dispersion (scale) are specified
scipy.stats.norm(loc = math.log(3.25), scale = 0.516).cdf(math.log(8))

0.9595693529448822

In [35]:
# write a function to return p-values when given a tephra load

def damage_ratio(t_load, mean, disp):
    return scipy.stats.norm(loc = math.log(mean), scale = disp).cdf(math.log(t_load))

In [36]:
data= {'tephra_load':[1,2,3,4,5,6,7,8,9,10],
       'n_buildings':[10000, 200, 3000, 40, 500, 60, 7000, 80, 9, 10]}

df = pd.DataFrame(data)
df

,tephra_load,n_buildings
0,1,10000
1,2,200
2,3,3000
3,4,40
4,5,500
5,6,60
6,7,7000
7,8,80
8,9,9
9,10,10


In [37]:
# Calculate damage ratio for each row based on tephra load

# Create an empty column for the DataFrame where values will be stored    
new_column = "dratio"
df[new_column] = None

# Iterate over the rows
for idx, row in df.iterrows():
    
    # Calculate the damage ratio
    drat = damage_ratio(t_load = row['tephra_load'], mean = 3.25, disp = 0.516)
    
    # Update the value of 'Celsius' column with the converted value
    df.at[idx, new_column] = drat
    
df

,tephra_load,n_buildings,dratio
0,1,10000,0.011179
1,2,200,0.173376
2,3,3000,0.438363
3,4,40,0.656306
4,5,500,0.798099
5,6,60,0.88262
6,7,7000,0.931483
7,8,80,0.959569
8,9,9,0.975808
9,10,10,0.985303


In [38]:
# Calculate repair costs for each row 

# Start by accessing the replacement cost from the 'vuln' dataframe
df['rep_cost_total'] = vuln['Total replacement cost (USD)'].loc[0]

# Multiply the total by the damage ratio
df["loss"] = df["dratio"]* df["rep_cost_total"].astype(float)

In [52]:
print('Total repair cost of all eruptions in analysis: $', int(sum(df['loss'])/1000), 'thousand USD')


Total repair cost of all eruptions in analysis: $ 104 thousand USD
